<h1 style="text-align: center; font-weight: bold; font-size: 36px;">Makemore Part 4: Backprop Ninja</h1>

# Introduction

Manual backprop through a **MLP** model. Pen-and-paper derivations

Inspired by Karpathy [Neural Networks: Zero-to-Hero](https://github.com/karpathy/nn-zero-to-hero). 
We are using the same [names.txt](https://github.com/karpathy/makemore/blob/master/names.txt) as in Zero to Hero so we can compare results.

# Imports

In [1]:
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Build the Dataset

In [2]:
with open('../data/names.txt', 'r') as f:
    names = f.read().splitlines()
print("Num names:", len(names))
print("Example names:", names[:10])
print("Min length:", min(len(name) for name in names))
print("Max length:", max(len(name) for name in names))

Num names: 32033
Example names: ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']
Min length: 2
Max length: 15


In [3]:
# Get vocabulary
letters = sorted(list(set(''.join(names))))
letters = ['.'] + letters
n_vocab = len(letters)
print(letters)

['.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
class Tokenizer:
    def __init__(self, vocab):
        assert isinstance(vocab, list)
        assert all(isinstance(v, str) for v in vocab)
        assert all(len(v) == 1 for v in vocab)
        self.stoi = {ch: i for i, ch in enumerate(vocab)}
        self.itos = {i: ch for i, ch in enumerate(vocab)}

    def encode(self, text):
        return [self.stoi[s] for s in text]

    def decode(self, sequence):
        if isinstance(sequence, list):
            return ''.join([self.itos[i] for i in sequence])
        elif isinstance(sequence, torch.Tensor):
            assert sequence.ndim in [0, 1]
            if sequence.ndim == 0:
                return self.itos[sequence.item()]  # one char
            else:
                return ''.join([self.itos[i.item()] for i in sequence])
        else:
            raise ValueError(f"Type {type(sequence)} not supported")

In [5]:
def build_dataset(tok, block_size, names):
    X, Y = [], []  # inputs and targets
    for name in names:
        name = '.'*block_size + name + '.'  # add start/stop tokens '..emma.'
        for i in range(len(name) - block_size):
            X.append(tok.encode(name[i:i+block_size]))
            Y.append(tok.encode(name[i+block_size])[0])  # [0] to keep Y 1d tensor
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

In [6]:
block_size = 3  # context length
tok = Tokenizer(vocab=letters)

random.seed(42)
random.shuffle(names)
n1 = int(0.8*len(names))
n2 = int(0.9*len(names))

Xtr, Ytr = build_dataset(tok, block_size, names[:n1])
Xval, Yval = build_dataset(tok, block_size, names[n1:n2])
Xtest, Ytest = build_dataset(tok, block_size, names[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [152]:
def cmp(s, at, bt):
    ex = torch.all(at == bt).item()
    app = torch.allclose(at, bt)
    maxdiff = (at - bt).abs().max().item()
    print(f'{s:15s} | exat: {str(ex):5s} | approx: {str(app):5s} | maxdiff: {maxdiff}')

In [138]:
# Init Layers
# NOTE: PyTorch F.cross_entropy uses multiple optimizations casuing slight
# numerical differences. This "lucky seed" makes our manual and PyTorch losses match.
torch.manual_seed(5)

# Hyperparameters
n_batch = 32
n_embd = 10
n_hid = 64

# Model
# NOTE: Init all params to non-zero so we don't mask gradient issues
C = torch.randn((n_vocab, n_embd))                              # n_vocab, n_emb (embeddings)
W1_kaiming_init = (5/3)/((n_embd*block_size)**0.5)              # tanh_gain / sqrt(fan_in)
W1 = torch.randn((n_embd*block_size, n_hid)) * W1_kaiming_init  # n_seq*n_emb, n_hid
b1 = torch.randn(n_hid)                      * 0.1              # n_hid
bngain = torch.ones((1, n_hid))              * 0.1 + 1.0
bnbias = torch.zeros((1, n_hid))             * 0.1
W2 = torch.randn((n_hid, n_vocab))           * 0.1              # n_hid, n_out
b2 = torch.randn(n_vocab)                    * 0.1              # 1, n_out

# Gather Params
params = [C, W1, b1, bngain, bnbias, W2, b2]
for p in params:
    p.requires_grad = True

# Random mini batch
batch_indices = torch.randint(0, Xtr.shape[0], (n_batch,))
x_batch = Xtr[batch_indices]
y_batch = Ytr[batch_indices]

In [151]:
# Forward Pass

# Embedding
emb = C[x_batch]                                  # n_batch, n_seq, n_emb
embcat = emb.view(-1, n_embd*block_size)          # n_batch, n_embd*block_size

# Linear 1
z1 = embcat @ W1 + b1                             # n_batch, n_hid

# Bachnorm
# zx = (x - x_mean) / (x_var + 1e-5)**0.5
# bn(x) = zx * gain + bias

# Batchnorm - mean
z1_sum = z1.sum(dim=0, keepdim=True)              # 1, n_hid
z1_mean = z1_sum / n_batch                        # 1, n_hid
# Batchnorm - var
z1_diff = z1 - z1_mean                            # n_batch, n_hid
z1_diff_2 = z1_diff**2.0
z1_diff_sum = z1_diff_2.sum(dim=0, keepdim=True)  # 1, n_hid
z1_var = z1_diff_sum / (n_batch-1)                # 1, n_hid
# Batchnorm - correction
z1_var_sqrt_inv = (z1_var + 1e-5)**-0.5
zx = z1_diff * z1_var_sqrt_inv                        # n_batch, n_hid

# Batchnorm - scaling
zz = bngain * zx + bnbias                         # n_batch, n_hid

# Linear 1 - tanh
# tanh(x) = (torch.exp(2*x) - 1.0) / (torch.exp(2*x) + 1)
zz_2 = 2*zz
zz_2_exp = zz_2.exp()
zz_2_exp_m1 = zz_2_exp - 1.0
zz_2_exp_p1 = zz_2_exp + 1.0
h1 = zz_2_exp_m1 / zz_2_exp_p1                    # n_batch, n_hid

# Linear 2
logits = h1 @ W2 + b2                       # n_batch, n_vocab

# Softmax
logits_max = logits.max(dim=-1, keepdim=True).values
logits_diff = logits - logits_max
logits_exp = logits_diff.exp()
logits_exp_sum = logits_exp.sum(dim=-1, keepdim=True)
logits_exp_sum_inv = logits_exp_sum**-1
probs = logits_exp * logits_exp_sum_inv  # probs

# Cross Entropy
log_probs = probs.log()
log_probs_target = log_probs[range(n_batch), y_batch]
loss = -log_probs_target.sum(dim=0, keepdim=True) / n_batch

# PyTorch Backward Pass
for p in params:
    p.grad = None
for t in [
        emb, embcat,
        z1,
        z1_sum, z1_mean,
        z1_diff, z1_diff_2, z1_diff_sum, z1_var,
        z1_var_sqrt_inv, zx,
        zz,
        zz_2, zz_2_exp, zz_2_exp_m1, zz_2_exp_p1, h1,
        logits,
        logits_max, logits_diff, logits_exp, logits_exp_sum, logits_exp_sum_inv, probs,
        log_probs, log_probs_target,
    ]:
    t.retain_grad()

loss.backward()

with torch.no_grad():
    # PyTorch loss: 3.473447322845459
    print(f"PyTorch loss: {F.cross_entropy(logits, y_batch).item()}")
    print(f"              {loss.item()}")

PyTorch loss: 3.473447322845459
              3.473447322845459


In [ ]:
# cmp("emb", d_emb, emb.grad)
# cmp("embcat", d_embcat, embcat.grad)
# cmp("z1", d_z1, z1.grad)
# cmp("z1_sum", d_z1_sum, z1_sum.grad)
# cmp("z1_mean", d_z1_mean, z1_mean.grad)
# cmp("z1_diff", d_z1_diff, z1_diff.grad)
# cmp("z1_diff_2", d_z1_diff_2, z1_diff_2.grad)
# cmp("z1_diff_sum", d_z1_diff_sum, z1_diff_sum.grad)
# cmp("z1_var", d_z1_var, z1_var.grad)
# cmp("z1_var_sqrt_inv", d_z1_var_sqrt_inv, z1_var_sqrt_inv.grad)
# cmp("zx", d_zx, zx.grad)
# cmp("zz", d_zz, zz.grad)
# cmp("zz_2", d_zz_2, zz_2.grad)
# cmp("zz_2_exp", d_zz_2_exp, zz_2_exp.grad)
# cmp("zz_2_exp_m1", d_zz_2_exp_m1, zz_2_exp_m1.grad)
# cmp("zz_2_exp_p1", d_zz_2_exp_p1, zz_2_exp_p1.grad)
# cmp("h1", d_h1, h1.grad)
# cmp("logits", d_logits, logits.grad)
# cmp("logits_max", d_logits_max, logits_max.grad)
# cmp("logits_diff", d_logits_diff, logits_diff.grad)
# cmp("logits_exp", d_logits_exp, logits_exp.grad)
# cmp("logits_exp_sum", d_logits_exp_sum, logits_exp_sum.grad)
# cmp("logits_exp_sum_inv", d_logits_exp_sum_inv, logits_exp_sum_inv.grad)
# cmp("probs", d_probs, probs.grad)
# cmp("log_probs", d_log_probs, log_probs.grad)
# cmp("log_probs_target", d_log_probs_target, log_probs_target.grad)